# TODO

- ~~Convert classes from text to tensor array with size 100~~
- ~~Complete train loop (it is not working now) due to problem above~~
- Find good LSTM architecture
- Check how fast is our approach with per frame check
- ~~Check if we can put videos with different frame number~~

## Device Allocation

In [1]:
import torch

#Set up device
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
use_cuda

cuda


True

## Dataset

In [2]:
import json
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [3]:
# Create a dataframe dataset object with all videos from .json config

def create_dataframe(json_config, videos_root):
    data = []
    videos = json.load(open(json_config))
    # train, val, test = dict(), dict(), dict()
    for name in os.listdir(videos_root):
        name = name[:-4]
        if videos.get(name) is None:
            continue
        video = videos[name]
        data.append([name, video['subset'], video['action'][1], video['action'][2], video['action'][0]])
    return pd.DataFrame(data, columns=['video_name', 'subset', 'start_frame', 'end_frame', 'class_id'])


# dataset = create_dataframe('output3.json', 'wlasl_dataset/videos')
dataset = create_dataframe('wlasl_dataset/nslt_100.json', 'wlasl_dataset/videos')
dataset

,video_name,subset,start_frame,end_frame,class_id
0,00623,train,1,104,51
1,00624,train,1,93,51
2,00625,test,1,35,51
3,00626,val,1,46,51
4,00627,val,1,28,51
...,...,...,...,...,...
1008,69524,val,1,59,74
1009,69531,val,1,54,30
1010,69533,val,1,64,49
1011,69534,train,1,54,7


In [4]:
from sklearn.preprocessing import OneHotEncoder

# Train one hot encoder to encoded classes from our dataset
all_classes = dataset["class_id"].to_list()
ohe = OneHotEncoder()
ohe.fit(np.array(all_classes).reshape(-1, 1))

# Test
# ohe.transform([[402]]).toarray()

OneHotEncoder()

In [5]:
from os.path import exists
import torch.nn.functional as F


class ASLDataset(Dataset):
    """
    A custom dataset class for loading American Sign Language (ASL) videos and their corresponding labels.

    Args:
        tensors_folder (str): Path to the folder containing the .pt arrays (converted videos).
        dataset_df (DataFrame): Pandas dataframe with information about videos
        max_frame_count (int): Maximum length of video in frames.
        ohe (OneHotEncoder): Required to encode class id to one hot numeric array.
    """

    def __init__(self, tensors_folder: str, dataset_df: pd.DataFrame, max_frame_count: int, ohe: OneHotEncoder) -> None:
        # Default frame (set of points) to insert in short videos to make all videos equal in length
        default_frame = [0] * (21 * 3 * 2 + 33 * 3)

        self.tensor_sequences = []
        self.targets = []
        indexes_to_drop = []
        for index, row in dataset_df.iterrows():
            path_to_tensor = tensors_folder + '/' + row["video_name"] + '.pt'

            if not exists(path_to_tensor):
                indexes_to_drop.append(index)
                continue

            # Open the video file using OpenCV
            tensor = torch.load(path_to_tensor).to(device)

            # Enhance frames(tensor) to the default length. [old version]: Change length of video (not a video but sequence of frames with points coordinates in each frame)
            pad_right = [default_frame for _ in range(0, max_frame_count - tensor.shape[0])]
            pad_right = torch.tensor(pad_right).to(device)
            self.tensor_sequences.append(torch.cat((F.normalize(tensor.float(), dim=1), pad_right)).to(device))
            # self.tensor_sequences.append(F.normalize(tensor.float(), dim=1))
        dataset_df.drop(index=indexes_to_drop, inplace=True)
        self.targets = torch.tensor(ohe.transform(np.array(dataset_df["class_id"]).reshape(-1, 1)).toarray()).to(device)

    def __len__(self):
        """
        Returns the number of videos in the dataset.

        Returns:
            int: The number of videos in the dataset.
        """
        return len(self.targets)

    def __getitem__(self, index):
        """
        Retrieves a video and its corresponding label from the dataset.

        Args:
            index (int): The index of the video to retrieve.

        Returns:
            tuple: A tuple containing the video frames as a PyTorch tensor and the label.
        """

        return self.tensor_sequences[index].float().to(device), self.targets[index].float().to(device)


In [6]:
def read_classes(path):
    classes = dict()
    with open(path, 'r') as file:
        for line in file:
            line = line.strip().split('\t')
            key = int(line[0])
            value = line[1]
            classes[key] = value
    return classes

In [7]:
max_frame_count = max(dataset["end_frame"] - dataset["start_frame"] + 1)
print(f"Maximum length of video in frames in current dataset is {max_frame_count}")

Maximum length of video in frames in current dataset is 203


In [8]:
train = dataset[dataset["subset"].isin(["train", 'val'])]
#val = dataset[dataset["subset"] == "val"]
test = dataset[dataset["subset"] == "test"]
classes = read_classes('wlasl_dataset/wlasl_class_list.txt')

train_dataset = ASLDataset('point_arrays', train.copy(), max_frame_count, ohe)
#val_dataset = ASLDataset('custom_dataset/point_arrays', val, max_frame_count, ohe)
test_dataset = ASLDataset('point_arrays', test.copy(), max_frame_count, ohe)

In [9]:
# Create dataloaders

batch_size = 16

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Train size: {len(train_dataset)}")
#print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")

Train size: 913
Test size: 100


## Model

In [10]:
import torch.nn as nn


class SequenceModel(nn.Module):
    def __init__(self, input_size=1662, output_size=100) -> None:
        super().__init__()
        self.lstm = nn.Sequential(
            nn.LSTM(input_size, 64, 1, batch_first=True),
        )
        self.linear = nn.Sequential(
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, output_size),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x[:, -1, :])

        return x


class LstmModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes) -> None:
        super(LstmModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, num_classes),
            # nn.Softmax(dim=1)
        )

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, 0, :])
        # out = self.fc(torch.mean(out, 1))
        return out


class GruModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes) -> None:
        super(GruModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, num_classes),
        )

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, 0, :])
        return out

In [11]:
# model = SequenceModel(input_size=225, output_size=2)
# model

## Training

In [12]:
from sklearn.metrics import f1_score
from tqdm import tqdm


def train_model(model: nn.Module, epochs: int, criterion, optimizer):
    """
    Function that trains model using number of epochs, loss function, optimizer.
    Can use validation or test data set for evaluation.
    Calculates f1 score.

    Parameter
    ---------
    model : nn.Module
      Model to train.
    epochs: int
      Number of train epochs
    criterion
      The loss function from pytorch
    optimizer
      The optimizer from pytorch
    """

    # Train the model
    for epoch in range(epochs):
        model.train()

        predicted_train = []
        true_train = []

        train_loss = 0.0

        bar = tqdm(train_dataloader)
        iterations = 0

        for inputs, outputs in bar:
            # Zero the gradients
            optimizer.zero_grad()
            inputs, outputs = inputs.to(device), outputs.to(device)

            # Forward pass
            predictions = model(inputs).to(device)
            loss = criterion(predictions, outputs)
            train_loss += loss.item()

            # Backward pass
            loss.backward()
            optimizer.step()

            # Get predicted classes and true classes from data
            for item in predictions:
                predicted_train.append(int(torch.argmax(item).cpu().numpy()))
            for item in outputs:
                true_train.append(int(torch.argmax(item).cpu().numpy()))
            iterations += 1
            bar.set_postfix(({"loss": f"{train_loss / (iterations * train_dataloader.batch_size)}"}))

        # Computing loss
        train_loss /= len(train_dataset)
        # Computing f1 score
        train_f1 = f1_score(true_train, predicted_train, average="macro")

        # Printing information in the end of train loop
        test_loss, test_f1 = test_model(model, criterion, test_dataloader)
        print(f"Epoch {epoch + 1} train (loss: {train_loss:.4f}, f1 score: {train_f1:.4f}) test (loss: {test_loss:.4f}, f1 score: {test_f1:.4f})")


def test_model(model: nn.Module, criterion, test_dataloader: DataLoader):
    """
    Function that evaluates model on specified dataloader
    by specified loss function.

    Parameter
    ---------
    model : nn.Module
      Model to train.
    criterion
      The loss function from pytorch
    test_dataloader: DataLoader
      The dataset for testing model

    Returns
    -------
    float: loss of model on given dataset
    float: f1 score of model on given dataset
    """

    model.eval()

    # Test loss value
    test_loss = 0.0

    # Lists for calculation f1 score
    predicted_test = []
    true_test = []

    with torch.no_grad():
        for inputs, outputs in test_dataloader:

            # Forward pass
            predictions = model(inputs.to(device))
            test_loss += criterion(predictions, outputs.to(device))

            # Get predicted classes and true classes from data
            for item in predictions:
                predicted_test.append(int(torch.argmax(item).cpu().numpy()))
            for item in outputs:
                true_test.append(int(torch.argmax(item).cpu().numpy()))

    # Computation of test loss
    test_loss /= len(test_dataloader)

    # Computation of f1 score
    test_f1 = f1_score(true_test, predicted_test, average="macro")
    return test_loss.item(), test_f1


In [13]:
sequence_size = max_frame_count
input_size = 21 * 3 * 2 + 33 * 3
hidden_size = 200
num_layers = 20
num_classes = len(ohe.categories_[0])
epochs = 100

# model = SequenceModel(input_size=225, output_size=num_classes).to(device)
# model = LstmModel(input_size, hidden_size, num_layers, num_classes).to(device)
model = GruModel(input_size, hidden_size, num_layers, num_classes).to(device)
# model = TestModel(input_size=225, sizes=[512, 256, 128, 128, num_classes]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

train_model(model, epochs, criterion, optimizer)

100%|██████████| 58/58 [00:19<00:00,  3.01it/s, loss=0.28800940462227526]


Epoch 1 train (loss: 0.2927, f1 score: 0.0007) test (loss: 4.6027, f1 score: 0.0003)


100%|██████████| 58/58 [00:18<00:00,  3.19it/s, loss=0.2878838279123964] 


Epoch 2 train (loss: 0.2926, f1 score: 0.0013) test (loss: 4.6039, f1 score: 0.0000)


100%|██████████| 58/58 [00:18<00:00,  3.20it/s, loss=0.2875241205610078] 


...

Epoch 5 train (loss: 0.2917, f1 score: 0.0006) test (loss: 4.6172, f1 score: 0.0005)


100%|██████████| 58/58 [00:18<00:00,  3.22it/s, loss=0.28707647426375027]


Epoch 6 train (loss: 0.2918, f1 score: 0.0003) test (loss: 4.6079, f1 score: 0.0005)


 28%|██▊       | 16/58 [00:05<00:14,  2.91it/s, loss=0.2864913996309042] 


KeyboardInterrupt: 

## Save the model to file

In [20]:
torch.save(model, f"model_gru.pt")

## Load model from file and train

In [21]:
model = torch.load('model_gru.pt')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
train_model(model, 35, criterion, optimizer)

 68%|██████▊   | 155/229 [00:50<00:24,  3.08it/s, loss=1.1439564620294878]


KeyboardInterrupt: 